In [14]:


conversionllm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

template = """Convert the following user queries into declarative sentences to enhance the effectiveness of a similarity search over a document folder. The conversion should reformulate the question into a statement that encapsulates the essence of the query, aiding in pinpointing the relevant documents. Below are a few examples to demonstrate the desired transformation:

User Query: Find a case law where fashion photographers were accused of copyright infringement.
Conversion: Fashion photographers were accused of copyright infringement.

User Query: Find the PDF where the defense to copyright infringement was freedom of expression.
Conversion: The defense to copyright infringement was freedom of expression.

User Query: Find proceeding where the defendants alleged entrapment against the plaintiffs.
Conversion: The defendants alleged entrapment against the plaintiffs.

User Query: In which judgements of copyright infringement ignorance was a defense that was used by the defendant?
Conversion: Copyright infringement where ignorance was a defense that was used by the defendant.

Now, convert the following user query:

User Query: {query}
Conversion: """
conversionPrompt = PromptTemplate.from_template(template)
pr=conversionPrompt.format(query=input_string)
finalquery=conversionllm.predict(pr)



Convert the following user queries into declarative sentences to enhance the effectiveness of a similarity search over a document folder. The conversion should reformulate the question into a statement that encapsulates the essence of the query, aiding in pinpointing the relevant documents. Below are a few examples to demonstrate the desired transformation:

User Query: Find a case law where fashion photographers were accused of copyright infringement.
Conversion: Fashion photographers were accused of copyright infringement.

User Query: Find the PDF where the defense to copyright infringement was freedom of expression.
Conversion: The defense to copyright infringement was freedom of expression.

User Query: Find proceeding where the defendants alleged entrapment against the plaintiffs.
Conversion: The defendants alleged entrapment against the plaintiffs.

User Query: In which judgements of copyright infringement ignorance was a defense that was used by the defendant?
Conversion: Copyr

In [ ]:

import nest_asyncio

nest_asyncio.apply()
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

import pandas as pd


from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.llms import OpenAI
hyde = HyDEQueryTransform(include_original=True)
llm = OpenAI(model="gpt-3.5-turbo")
query_bundle = hyde.run("Find cases where the defendants claimed that they were tricked by the cops")
query=query_bundle.custom_embedding_strs[0]+'\n  '+query_bundle.custom_embedding_strs[1]


In [26]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document

from langchain.document_loaders import UnstructuredPDFLoader
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

def pdf_to_text(pdf_file_path, dpi=200, pages=None):
    images = convert_from_path(pdf_file_path, dpi=dpi)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image)
    return text

dir_path = './media1/media2'
books=[]
filenames=[]
for root, dirs, files in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)

        
        extracted_text = pdf_to_text(file_path)
        text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4*15000,
        chunk_overlap=4*100,
        length_function=len,
        )

        chunks = text_splitter.create_documents([extracted_text])


    


        filenames.append(file)
        books.append(chunks)

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
import os
from langchain.prompts import PromptTemplate
print(books)


In [69]:


llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")

prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
results=[]
for book in books:
    result=chain({"input_documents": book}, return_only_outputs=True)
    results.append(result['output_text'])

In [ ]:
summary_docs=[]
for text, title in zip(results,filenames):
    pass



In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [103]:
vectorizer = TfidfVectorizer()
document_vectors = vectorizer.fit_transform(results)
query_vector = vectorizer.transform([query])
similarity_scores = cosine_similarity(query_vector, document_vectors)
top_k_indices = similarity_scores.argsort()[0][-1:]
top_k_documents = [results[i] for i in top_k_indices]
answer=top_k_indices[0]
print(filenames[answer])
print(results[answer])

393.pdf
In the case of Volkov and Adamskiy v. Russia, the applicants were convicted of copyright infringement after undercover police officers contacted them and asked them to install unlicensed software on their computers. The applicants argued that they were entrapped by the police, violating their right to a fair trial. The Court found that the applicants had willingly installed the software without any explicit request or unlawful incitement from the police, so their claim of entrapment was inadmissible. However, the Court also found that Mr. Volkov was not provided with legal assistance during his appeal hearing, which violated his right to a fair trial. The Court awarded him EUR 4,000 in non-pecuniary damages.


Information Note on the Court's case-law No. 159
January 2013

Ashby Donald and Others v. France - 36769/08
Judgment 10.1.2013 [Section V]

Article 10
Article 10-1

Freedom of expression

Conviction of photographers for copyright infringement through publication on the
Internet of photographs of fashion show: no violation

ARTICLE 10
Freedom of expression
no violation

Ashby Donald and Others v. France - 36769/08
Judgment 10.1.2013 [Section V]

Facts - The applicants are fashion photographers. Accredited by the French
designers’ federation Fédération francaise de la couture for different fashion
publications, they were invited by various fashion houses to the women’s winter
2003/2004 collection fashion shows in March 2003. They had not signed any
exclusive agreements. Photographs they took at the fashion shows were sent to a
company that published them on line, a few hours after the shows, on a
specialised Web site offering photos and videos of fashion shows on a free or pay-
to-view b

In [ ]:


vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)



query_engine = vector_index.as_query_engine()

In [47]:
eval_questions = [

    "",

]
responses = []

# Loop through each question and get a response
for question in eval_questions:
    response = query_engine.query(question)
    responses.append(response)



In [48]:
responses = [a.response for a in responses]
print(responses)

['Manhattan is the borough of New York City that has the highest population.']


In [49]:

ground_truths = [
 
    "",  
]
contexts=['',]
contexts=[[a] for a in contexts]

ground_truths = [[a] for a in ground_truths]

from datasets import Dataset 

data_samples = {
    'question': eval_questions,
    'answer': responses,
    'contexts': contexts,
    'ground_truths': ground_truths
}
dataset = Dataset.from_dict(data_samples)



In [50]:
print(data_samples)

{'question': ['Which borough of New York City has the highest population?'], 'answer': ['Manhattan is the borough of New York City that has the highest population.'], 'contexts': [['']], 'ground_truths': [['Manhattan']]}


In [51]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas import evaluate

result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

evaluating with [context_precision]


100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.87s/it]


evaluating with [faithfulness]


100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.10s/it]


evaluating with [answer_relevancy]


100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.41s/it]


evaluating with [context_recall]


100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.17s/it]


{'context_precision': 0.0000, 'faithfulness': 0.5000, 'answer_relevancy': 1.0000, 'context_recall': 0.0000}

In [28]:
df = result.to_pandas()
df.head()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
filename = "a111.xlsx"


df.to_excel(filename, index=False) 

In [15]:
type(result.to_pandas())
table=result.to_pandas()